In [ ]:
import pygame
from pygame.locals import *
import random
import time

pygame.init()
FPS = 120
FramePerSec = pygame.time.Clock()

SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
SPEED = 5
SCORE = 0
COINS_COLLECTED = 0
PREV_MILESTONE = 0 

coin_sound = pygame.mixer.Sound("coin.mp3")

DISPLAY = pygame.display.set_mode((400, 600))
DISPLAY.fill((255, 255, 255))
pygame.display.set_caption("GAME")

font = pygame.font.SysFont("Verdana", 60)
font_small = pygame.font.SysFont("Times New Roman", 20)
game_over = font.render("Game Over", True, (0, 0, 0))

background = pygame.image.load("AnimatedStreet.png")
pygame.mixer.music.load("background.wav")
pygame.mixer.music.play(-1)

class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("Enemy.png")
        self.image = pygame.transform.scale(self.image, (50, 100))
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(20, SCREEN_WIDTH - 20), 0)

    def move(self):
        global SCORE
        self.rect.move_ip(0, SPEED)
        if self.rect.bottom > 600:
            SCORE += 1
            self.rect.top = 0
            self.rect.center = (random.randint(20, SCREEN_WIDTH - 20), 0)

class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load("Player.png")
        self.image = pygame.transform.scale(self.image, (50, 100))
        self.rect = self.image.get_rect()
        self.rect.center = (207, 520)

    def move(self):
        pressed_keys = pygame.key.get_pressed()
        if self.rect.left > 0 and pressed_keys[K_LEFT]:
            self.rect.move_ip(-5, 0)
        if self.rect.right < SCREEN_WIDTH and pressed_keys[K_RIGHT]:
            self.rect.move_ip(5, 0)

class Coin(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()
        self.respawn()

    def respawn(self):
        weights = [1, 2, 3]  
        self.weight = random.choice(weights) 
        image_path = f"coin{self.weight}.jpg"
        self.original_image = pygame.image.load("coin1.png")
        size = 30 + self.weight * 5 
        self.image = pygame.transform.scale(self.original_image, (size, size))
        self.rect = self.image.get_rect()
        self.rect.center = (random.randint(20, SCREEN_WIDTH - 20), 0)

    def move(self):
        self.rect.move_ip(0, SPEED // 2)
        if self.rect.top > SCREEN_HEIGHT:
            self.respawn()

P1 = Player()
E1 = Enemy()
C1 = Coin()

enemies = pygame.sprite.Group()
enemies.add(E1)

coins = pygame.sprite.Group()
coins.add(C1)

all_sprites = pygame.sprite.Group()
all_sprites.add(P1, E1, C1)

INC_SPEED = pygame.USEREVENT + 1
pygame.time.set_timer(INC_SPEED, 1000)

while True:
    for event in pygame.event.get():
        if event.type == INC_SPEED:
            SPEED += 0.3
        if event.type == QUIT:
            pygame.quit()

    DISPLAY.blit(background, (0, 0))
 
    scores = font_small.render(str(SCORE), True, (0, 0, 0))
    DISPLAY.blit(scores, (10, 10))
    coins_collected_text = font_small.render(f"Coins: {COINS_COLLECTED}", True, (0, 0, 0))
    DISPLAY.blit(coins_collected_text, (300, 10))
    
    for entity in all_sprites:
        entity.move()
        DISPLAY.blit(entity.image, entity.rect)
        
    if pygame.sprite.spritecollideany(P1, enemies):
        pygame.mixer.music.stop()
        pygame.mixer.Sound("crash.wav").play()
        time.sleep(0.7)
        DISPLAY.fill((255, 0, 0))
        DISPLAY.blit(game_over, (30, 250))
        pygame.display.update()
        for entity in all_sprites:
            entity.kill()
        time.sleep(1)
        pygame.quit()

    if pygame.sprite.spritecollideany(P1, coins):
        COINS_COLLECTED += C1.weight
        coin_sound.play()
        C1.respawn()
        if COINS_COLLECTED // 10 > PREV_MILESTONE:
            SPEED += 0.5
            PREV_MILESTONE = COINS_COLLECTED // 10

    pygame.display.update()
    FramePerSec.tick(FPS)


In [ ]:
import pygame  
import sys  
import copy  
import random  
import time  

pygame.init()  

scale = 15  
score = 0  
level = 0  
SPEED = 10  

food_x = 10  
food_y = 10  

display = pygame.display.set_mode((500, 500))
pygame.display.set_caption("Snake Game")  
clock = pygame.time.Clock()  

background_top = (0, 0, 50)  
background_bottom = (0, 0, 0)  
snake_colour = (255, 137, 0)  
food_colour = (random.randint(1, 255), random.randint(1, 255), random.randint(1, 255))  
snake_head = (255, 247, 0)  
font_colour = (255, 255, 255)  
defeat_colour = (255, 0, 0)  

class Snake:
    def __init__(self, x_start, y_start):
        self.x = x_start  
        self.y = y_start  
        self.w = 15  
        self.h = 15  
        self.x_dir = 1  
        self.y_dir = 0  
        self.history = [[self.x, self.y]]  
        self.length = 1  

    def reset(self):
        self.x = 500 / 2 - scale  
        self.y = 500 / 2 - scale  
        self.w = 15  
        self.h = 15  
        self.x_dir = 1  
        self.y_dir = 0  
        self.history = [[self.x, self.y]]  
        self.length = 1  

    def show(self):
        for i in range(self.length):
            color = snake_head if i == 0 else snake_colour
            pygame.draw.rect(display, color, (self.history[i][0], self.history[i][1], self.w, self.h))

    def check_eaten(self):
        if abs(self.history[0][0] - food_x) < scale and abs(self.history[0][1] - food_y) < scale:
            return True

    def check_level(self):
        global level
        if self.length % 5 == 0:
            return True

    def grow(self):
        self.length += 1
        self.history.append(self.history[self.length - 2])

    def death(self):
        for i in range(1, self.length):
            if abs(self.history[0][0] - self.history[i][0]) < self.w and abs(self.history[0][1] - self.history[i][1]) < self.h and self.length > 2:
                return True

    def update(self):
        for i in range(self.length - 1, 0, -1):
            self.history[i] = copy.deepcopy(self.history[i - 1])
        self.history[0][0] += self.x_dir * scale
        self.history[0][1] += self.y_dir * scale

class Food:
    def new_location(self):
        global food_x, food_y, food_colour
        food_x = random.randrange(1, int(500 / scale) - 1) * scale
        food_y = random.randrange(1, int(500 / scale) - 1) * scale
        food_colour = (random.randint(1, 255), random.randint(1, 255), random.randint(1, 255))

    def show(self):
        pygame.draw.rect(display, food_colour, (food_x, food_y, scale, scale))

def show_score():
    font = pygame.font.SysFont(None, 20)
    text = font.render("Score: " + str(score), True, font_colour)
    display.blit(text, (scale, scale))

def show_level():
    font = pygame.font.SysFont(None, 20)
    text = font.render("Level: " + str(level), True, font_colour)
    display.blit(text, (90 - scale, scale))

def gameLoop():
    global score
    global level
    global SPEED

    snake = Snake(500 / 2, 500 / 2)  
    food = Food()  
    food.new_location()  

    while True:  
        for event in pygame.event.get():  
            if event.type == pygame.QUIT:  
                pygame.quit()  
                sys.exit()  
            if event.type == pygame.KEYDOWN:  
                if event.key == pygame.K_q:  
                    pygame.quit()  
                    sys.exit()  
                if snake.y_dir == 0:  
                    if event.key == pygame.K_UP:  
                        snake.x_dir = 0  
                        snake.y_dir = -1  
                    if event.key == pygame.K_DOWN:  
                        snake.x_dir = 0  
                        snake.y_dir = 1  
                if snake.x_dir == 0:  
                    if event.key == pygame.K_LEFT:  
                        snake.x_dir = -1  
                        snake.y_dir = 0  
                    if event.key == pygame.K_RIGHT:  
                        snake.x_dir = 1  
                        snake.y_dir = 0  

        for y in range(500):
            color = (
                background_top[0] + (background_bottom[0] - background_top[0]) * y / 500,
                background_top[1] + (background_bottom[1] - background_top[1]) * y / 500,
                background_top[2] + (background_bottom[2] - background_top[2]) * y / 500
            )
            pygame.draw.line(display, color, (0, y), (500, y))

        snake.show()  
        snake.update()  
        food.show()  
        show_score()  
        show_level()  

        if snake.check_eaten():  
            food.new_location()  
            score += random.randint(1, 5)  
            snake.grow()  

        if snake.check_level():  
            food.new_location()  
            level += 1  
            SPEED += 1  
            snake.grow()  

        if snake.death():  
            score = 0  
            level = 0  
            font = pygame.font.SysFont(None, 100)  
            text = font.render("Game Over!", True, defeat_colour)  
            display.blit(text, (50, 200))  
            pygame.display.update()  
            time.sleep(3)  
            snake.reset()  

        if snake.history[0][0] > 500:  
            snake.history[0][0] = 0  
        if snake.history[0][0] < 0:  
            snake.history[0][0] = 500  
        if snake.history[0][1] > 500:
            snake.history[0][1] = 0
        if snake.history[0][1] < 0:
            snake.history[0][1] = 500

        pygame.display.update()
        clock.tick(SPEED)

gameLoop()
